In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
df = df.drop(['MiscFeature', 'Alley', 'PoolQC', 'Fence','LotFrontage', 'FireplaceQu' ], axis = 1)
df = df.dropna()

In [5]:
X = df[['Street', 'Utilities', 'YearBuilt','LotArea', 'PoolArea', 'YrSold']]
df.Street = df.Street.replace({'Pave':1, "Grvl":0})
df.Utilities = df.Utilities.replace({'AllPub':1, 'NoSeWa':0})

In [10]:
x = X[['YearBuilt', 'LotArea', 'PoolArea']]
x_mean = x.mean()
x_std = x.std()
X_st = (x-x_mean)/x_std
X = X_st
X.head()

,YearBuilt,LotArea,PoolArea
0,1.013778,-0.218282,-0.071744
1,0.100490,-0.107027,-0.071744
2,0.946127,0.052600,-0.071744
3,-1.962862,-0.111864,-0.071744
4,0.912301,0.343798,-0.071744


In [8]:
Y = (df.SalePrice - df.SalePrice.mean())/df.SalePrice.std()

### Model 1

In [11]:
X = sm.add_constant(X)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.326
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     214.7
Date:                Mon, 25 Mar 2019   Prob (F-statistic):          1.21e-113
Time:                        13:38:51   Log-Likelihood:                -1634.4
No. Observations:                1338   AIC:                             3277.
Df Residuals:                    1334   BIC:                             3298.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.274e-17      0.022  -1.46e-15      1.000      -0.044       0.044
YearBuilt      0.5055      0.022     22.483      0.000       0.461       0.550
LotArea        0.2518      0.023     11.164      0.000       0.208       0.296
PoolArea       0.0720      0.023      3.192      0.001       0.028       0.116
==============================================================================
Omnibus:                      656.483   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5593.666
Skew:                           2.102   Prob(JB):                         0.00
Kurtosis:                      12.091   Cond. No.                         1.08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Model 2 (özellik sayısı arttırılmış)

Şimdi satış fiyatıyla yüksek korelasyonu olduğunu tespit ettiğimiz özellikleri de modele dahil edip modelimizi tahmin edelim.

In [16]:
df.OverallQual = (df.OverallQual - df.OverallQual.mean()) / df.OverallQual.std()
df.GarageCars = (df.GarageCars - df.GarageCars.mean()) / df.GarageCars.std()
df.YearRemodAdd = (df.YearRemodAdd - df.YearRemodAdd.mean()) / df.YearRemodAdd.std()

In [17]:
X2 = df[['YearBuilt','LotArea', 'PoolArea', 'OverallQual', 'GarageCars', 'YearRemodAdd']]

X2 = sm.add_constant(X2)
results2 = sm.OLS(Y,X2).fit()
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.693
Model:                            OLS   Adj. R-squared:                  0.691
Method:                 Least Squares   F-statistic:                     500.1
Date:                Mon, 25 Mar 2019   Prob (F-statistic):               0.00
Time:                        13:48:23   Log-Likelihood:                -1108.6
No. Observations:                1338   AIC:                             2231.
Df Residuals:                    1331   BIC:                             2268.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.5321      1.418     -0.375      0.707      -3.313       2.249
YearBuilt        0.0002      0.001      0.252      0.801      -0.001       0.002
LotArea       1.607e-05    1.5e-06     10.731      0.000    1.31e-05     1.9e-05
PoolArea         0.0009      0.000      2.465      0.014       0.000       0.002
OverallQual      0.5866      0.021     27.762      0.000       0.545       0.628
GarageCars       0.2359      0.020     11.849      0.000       0.197       0.275
YearRemodAdd     0.0671      0.020      3.291      0.001       0.027       0.107
==============================================================================
Omnibus:                      589.490   Durbin-Watson:                   1.950
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9167.515
Skew:                           1.630   Prob(JB):                         0.00
Kurtosis:                      15.402   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

* Model 2 için 3 özellik daha eklemiştik. Adjusted R-Squared değerine baktığımızda neredeyse 2 katlık bir artış görüyoruz. 
* Yine Model 2'de AIC ve BIC değerleri de düştü. Bu açılardan Model 2 Model 1'den daha iyi. Ancak 'YearBuilt' değişkeninin P değerinin yüksek olduğunu görüyoruz. Bunu çıkartıp yeniden bir OLS uygulayalım.

### Model 3 (YearBuilt özelliği çıkarıldı)

In [18]:
X3 = df[['LotArea', 'PoolArea', 'OverallQual', 'GarageCars', 'YearRemodAdd']]

X3 = sm.add_constant(X3)
results3 = sm.OLS(Y,X3).fit()
results3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.693
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     600.5
Date:                Mon, 25 Mar 2019   Prob (F-statistic):               0.00
Time:                        13:57:51   Log-Likelihood:                -1108.7
No. Observations:                1338   AIC:                             2229.
Df Residuals:                    1332   BIC:                             2261.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.1744      0.022     -7.924      0.000      -0.218      -0.131
LotArea       1.604e-05   1.49e-06     10.747      0.000    1.31e-05     1.9e-05
PoolArea         0.0009      0.000      2.462      0.014       0.000       0.002
OverallQual      0.5879      0.021     28.652      0.000       0.548       0.628
GarageCars       0.2370      0.019     12.230      0.000       0.199       0.275
YearRemodAdd     0.0691      0.019      3.713      0.000       0.033       0.106
==============================================================================
Omnibus:                      588.117   Durbin-Watson:                   1.950
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9133.038
Skew:                           1.625   Prob(JB):                         0.00
Kurtosis:                      15.380   Cond. No.                     2.40e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.4e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

'YearBuilt' özelliğini attık. Görüyoruz ki Adjusted R Squared değeri aynı, aynı şekilde BIC ve AIC değerlerinde de bir değişim yok. Bu 'YearBuilt' değişkeninin artık bizim için o kadar önemli olmadığına işaret eder. Varyans zaten diğer değişkenler tarafından yeterince açıklanıyor. Son modelimizde bir kaç değişken daha ekleyelim. Bu arada katsayısı çok düşük olan 'LotArea' özelliğini de çıkaralım.

### Model 4 (Model 3'e yeni özellikler eklendi, LotArea çıkarıldı)

In [25]:
df.Fireplaces = (df.Fireplaces - df.Fireplaces.mean()) / df.Fireplaces.std()
df.GrLivArea = (df.GrLivArea - df.GrLivArea.mean()) / df.GrLivArea.std()
df.TotalBsmtSF = (df.TotalBsmtSF - df.TotalBsmtSF.mean()) / df.TotalBsmtSF.std()

In [28]:
X4 = df[['LotArea', 'PoolArea', 'OverallQual', 'GarageCars', 'YearRemodAdd', 'Fireplaces','GrLivArea','TotalBsmtSF']]
X4 = sm.add_constant(X4)
results4 = sm.OLS(Y,X4).fit()
results4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.770
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                     557.5
Date:                Mon, 25 Mar 2019   Prob (F-statistic):               0.00
Time:                        14:16:00   Log-Likelihood:                -913.59
No. Observations:                1338   AIC:                             1845.
Df Residuals:                    1329   BIC:                             1892.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0742      0.020     -3.765      0.000      -0.113      -0.036
LotArea       7.038e-06   1.37e-06      5.137      0.000    4.35e-06    9.73e-06
PoolArea        -0.0004      0.000     -1.203      0.229      -0.001       0.000
OverallQual      0.3619      0.021     17.429      0.000       0.321       0.403
GarageCars       0.1420      0.017      8.134      0.000       0.108       0.176
YearRemodAdd     0.1128      0.016      6.914      0.000       0.081       0.145
Fireplaces       0.0674      0.015      4.381      0.000       0.037       0.098
GrLivArea        0.2774      0.018     15.226      0.000       0.242       0.313
TotalBsmtSF      0.1576      0.016      9.606      0.000       0.125       0.190
==============================================================================
Omnibus:                      456.446   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            57169.530
Skew:                          -0.487   Prob(JB):                         0.00
Kurtosis:                      35.008   Cond. No.                     2.77e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.77e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

* Model 4'te Adjusted R Squared ve AIC-BIC değerlerinde iyileşme oldu ancak, P-değeri yüksek olan 'PoolArea' ve katsayısı çok düşük olan 'LotArea' özelliklerini çıkarıp tekrar tahmin yapalım.

### Model 5 (LotArea ve PoolArea çıkarıldı)

In [29]:
X5 = df[['OverallQual', 'GarageCars', 'YearRemodAdd', 'Fireplaces','GrLivArea','TotalBsmtSF']]
X5 = sm.add_constant(X5)
results5 = sm.OLS(Y,X5).fit()
results5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     724.7
Date:                Mon, 25 Mar 2019   Prob (F-statistic):               0.00
Time:                        14:18:28   Log-Likelihood:                -927.38
No. Observations:                1338   AIC:                             1869.
Df Residuals:                    1331   BIC:                             1905.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -3.274e-17      0.013  -2.47e-15      1.000      -0.026       0.026
OverallQual      0.3479      0.021     16.768      0.000       0.307       0.389
GarageCars       0.1467      0.018      8.358      0.000       0.112       0.181
YearRemodAdd     0.1108      0.016      6.727      0.000       0.078       0.143
Fireplaces       0.0801      0.015      5.228      0.000       0.050       0.110
GrLivArea        0.2875      0.018     15.968      0.000       0.252       0.323
TotalBsmtSF      0.1714      0.016     10.584      0.000       0.140       0.203
==============================================================================
Omnibus:                      457.794   Durbin-Watson:                   1.961
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            52285.491
Skew:                          -0.528   Prob(JB):                         0.00
Kurtosis:                      33.606   Cond. No.                         3.18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* Model 5'in P değeri 0'a yakın.
* Bütün özellikler istatistiksel olarak anlamlı görünüyor.
* Modeldeki özellikler varyansın 0.766'sını açıklıyor.
* Model 4'e göre daha az özellik içermesine rağmen Adjusted R-Squared değeri aynı kaldı. 
* Aynı şekilde AIC ve BIC parametrelerinde de gözle görülür bir değişme yok. 
* Sonuç olarak modellerimiz içinde en iyisi 'Model 5' diyebiliriz.